In [1]:
import os

import boto3

from config.settings import dev
from config.settings.base import CHROME_DRIVER_DIR
from config.settings.base import MEDIA_ROOT
from django.core.files import File

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import urllib.request


# class Starbucks:
#     def __init__(self, name, info, size, calorie, fat, protein, sodium, sugar, caffeine):
#         self.name = name
#         self.info = info
#         self.size = size
#         self.calorie = calorie
#         self.fat = fat
#         self.protein = protein
#         self.sodium = sodium
#         self.sugar = sugar
#         self.caffeine = caffeine

#     @classmethod
# def get_coffee_info():
chrome_options = Options()
# chrome_options.add_argument('headless')
# chrome_options.add_argument('disable-gpu')

if 'CHROMEDRIVER_VERSION' in os.environ:
    chrome_driver_linux = os.path.join(CHROME_DRIVER_DIR, 'chromedriver_linux')
    driver = webdriver.Chrome(chrome_driver_linux, chrome_options=chrome_options)
else:
    CHROME_DRIVER = os.path.join(CHROME_DRIVER_DIR, 'chromedriver')
    driver = webdriver.Chrome(CHROME_DRIVER, chrome_options=chrome_options)
driver.get('http://www.istarbucks.co.kr/menu/drink_list.do')
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
dt_category = soup.select('dl.product_view_tab > dd > div.product_list > dl > dt > a')
dd_content = soup.select('dl.product_view_tab > dd > div.product_list > dl > dd')

for a, b in zip(dt_category, dd_content):
    # 카테고리 타이틀
    category_title = a.get_text(strip=True)
    category, is_category = CoffeeCategory.objects.get_or_create(
        name=category_title
    )
    # 커피 이름 & 이미지 URL & 커피 디테일 URL
    coffee_name_image_list = b.select('ul > li.menuDataSet')
    for name_image in coffee_name_image_list:
        coffee_name = name_image.select_one('dl > dd').get_text()
        coffee_url_total = name_image.select_one('dl > dt > a')
        coffee_url = coffee_url_total.get('prod')
        coffee_base_url = 'http://www.istarbucks.co.kr/menu/drink_view.do?product_cd='

        # 커피 디테일 페이지 접근
        driver.get(coffee_base_url + coffee_url)
        driver.implicitly_wait(7)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        pages_detail = soup.select('div.content02 > div.product_view_wrap1 > div.product_view_detail')
        pages_detail2 = soup.select('form > fieldset > div.product_view_info > div.product_info_content')    
        character = soup.find_all('dd', text='-')
        if character:
            pass
        else:
            for detail_1, detail_2 in zip(pages_detail, pages_detail2):
                detail_names = detail_1.select_one('div.myAssignZone > h4').get_text(strip=True)
                print(detail_names)
                # 커피 detail info
                detail_infos = detail_1.select_one('div.myAssignZone > p').get_text(strip=True)
                print(detail_infos)
                # 커피 detail 제품영양정보 단위
                detail_sizes = detail_1.select_one(
                    'form > fieldset > div.product_view_info > div.product_info_head '
                    '> div.product_select_wrap2 > div.selectTxt2'
                ).get_text(strip=True)
                detail_kcal = detail_2.select_one('ul > li.kcal > dl > dd').get_text(strip=True)
                # 커피 detail sat_FAT
                detail_fats = detail_2.select_one('ul > li.sat_FAT > dl > dd').get_text(strip=True)
                # 커피 detail protein
                detail_proteins = detail_2.select_one('ul > li.protein > dl > dd').get_text(strip=True)
                # 커피 detail sodium
                detail_sodiums = detail_2.select_one('ul:nth-of-type(2) > li.sodium > dl > dd').get_text(
                    strip=True)
                # 커피 detail sugar
                detail_sugars = detail_2.select_one('ul:nth-of-type(2) > li.sugars > dl > dd').get_text(
                    strip=True)
                # 커피 detail caffeine
                detail_caffeines = detail_2.select_one('ul:nth-of-type(2) > li.caffeine > dl > dd').get_text(
                    strip=True)
                coffee, is_coffee = Coffee.objects.get_or_create(
                    coffeeshop_list='STARBUCKS',
                    category=category,
                    name=detail_names,
                    coffee_info=detail_infos,
                    coffee_size=detail_sizes,
                    calorie=detail_kcal,
                    saturated_fat=detail_fats,
                    protein=detail_proteins,
                    sodium=detail_sodiums,
                    sugars=detail_sugars,
                    caffeine=detail_caffeines,
                )
                STARBUCKS_DIR = os.path.join(MEDIA_ROOT, '.starbucks')
                STARBUCKS_IMAGE_DIR = os.path.join(STARBUCKS_DIR, f'{coffee_name}.jpg')
                if not os.path.exists(STARBUCKS_DIR):
                    os.makedirs(STARBUCKS_DIR, exist_ok=True)
                if is_coffee:    
                    try:
                        coffee_image_url = name_image.select_one('dl > dt > a > img')
                        coffee_image = coffee_image_url.get('src')
                        urllib.request.urlretrieve(coffee_image, STARBUCKS_IMAGE_DIR)
                        f = open(os.path.join(STARBUCKS_DIR, f'{coffee_name}.jpg'), 'rb')
                        image, is_image = CoffeeImage.objects.get_or_create(
                            location=File(f),
                            coffee=coffee,
                        )
                        f.close()
                    except FileExistsError:
                        print('이미 존재하는 파일')

driver.close()

/usr/local/var/pyenv/versions/coffee-calorie/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: use options instead of chrome_options


돌체 콜드 브루Dolce Cold Brew
무더운 여름철,동남아 휴가지에서 즐기는 커피를 떠오르게 하는스타벅스 음료의 베스트 x 베스트 조합인돌체 콜드 브루를 만나보세요!
나이트로 쇼콜라Nitro Chocolat
초콜릿과 견과류의 풍미, 초콜릿 파우더 토핑, 풀 바디감의 새로운 나이트로 콜드 브루
나이트로 쇼콜라 클라우드Nitro Chocolat Cloud
초콜릿과 견과류의 풍미, 초콜릿 파우더 토핑, 하프&하프 샷부드럽고 달콤한 새로운 나이트로 콜드 브루
나이트로 콜드 브루Nitro Cold Brew
나이트로 커피 정통의 캐스케이딩과 부드러운 콜드 크레마!부드러운 목 넘김과 완벽한 밸런스에 커피 본연의 단맛을 경험할 수 있습니다.
바닐라 크림 콜드 브루Vanilla Cream Cold Brew
콜드 브루에 더해진 바닐라 크림으로 깔끔하면서 달콤한 콜드 브루를 새롭게 즐길 수 있는 음료입니다.
콜드 브루Cold Brew
스타벅스 바리스타의 정성으로 탄생한 콜드 브루!콜드 브루 전용 원두를 차가운 물로 14시간 동안 추출하여 한정된 양만 제공됩니다.깊은 풍미의 새로운 커피 경험을 즐겨보세요.
콜드 브루 몰트Cold Brew Malt
[리저브 전용음료] 리저브 콜드 브루, 바닐라 아이스크림, 몰트가 블렌딩된 리저브만의 쉐이크
콜드 브루 플로트Cold Brew Float
[리저브 전용음료] 리저브 콜드 브루 위에 녹아 내리는 한 스쿱의 바닐라 아이스크림
콜드 폼 콜드 브루Cold Foam Cold Brew
부드럽고 쫀득하며 달콤한 콜드 폼을 즐기고 콜드 브루를 같이 마시면 더욱 부드러운 목넘김을 즐길 수 있는 음료 입니다.
아이스 커피Iced Coffee
깔끔하고 상큼함이 특징인 시원한 아이스 커피
오늘의 커피Brewed Coffee
신선하게 브루드(Brewed)되어 원두의 다양함이 살아있는 커피
블론드 리스트레토 마키아또Blonde Ristretto Macchiato
블론드 에스프레소의 달콤하고 시트러스한 풍미의 매력을 블론드 리스트레토 마키아또로 즐겨보세요!
블

KeyboardInterrupt: 

In [ ]:
if not is_follow: 
        follow.delete()
    else:
        return follow

In [ ]:
is_coffee